In [1]:
from selenium import webdriver
import pandas as pd
import os
import sys
from os import listdir
from os.path import isfile, join
import time
import datetime
from datetime import timedelta, date
import itertools
import random
import requests
from bs4 import BeautifulSoup
from random import choice
from fake_useragent import UserAgent

In [7]:
current_path = os.getcwd()
DRIVER_BIN = os.path.join(current_path, "chromedriver")
browser = webdriver.Chrome()

dep_airport = 'EWR'
arrival_airport = 'SIN'
dep_date = datetime.datetime(2022, 5, 1)
url = "https://www.expedia.com/Flights-Search?trip=oneway&leg1=from:{0},to:{1},departure:{2}TANYT&passengers=adults:1,children:0,seniors:0,infantinlap:Y&options=cabinclass%3Aeconomy&mode=search&origref=www.expedia.com".format(dep_airport,arrival_airport,dep_date.strftime("%m/%d/%Y"))
browser.get(url)
time.sleep(random.uniform(5,8)) #wait for the results to finish

In [ ]:
df = pd.DataFrame()
def compile_data(dep_date):
    global df
    global dep_times_list
    global arr_add_days_list
    global arr_dates_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list
    global departure_airport_code
    global arrival_airport_code
    
    dep_times_list = []
    arr_times_list = []
    arr_add_days_list = []
    arr_dates_list = []
    airlines_list = []
    price_list = []
    durations_list = []
    stops_list = []
    layovers_list = []
    departure_airport_code = []
    arrival_airport_code = []
    
    #departure times, arrival dates, arrival times
    everything = browser.find_elements_by_xpath("//div[@class='primary-content no-wrap custom-primary-padding']")
    to_parse = [value.text for value in everything]
for value in to_parse:
        parsed = value.split(' ')
        parsed[0] = parsed[0][:-2]
        dep_times_list.append(parsed[0])
        arr_times_list.append(parsed[1])
        if len(parsed) > 2:
            parsed[2] = parsed[2][:2]
            arr_add_days_list.append(int(parsed[2]))
        else:
            arr_add_days_list.append(0)
    
    arr_dates_list = [dep_date + timedelta(days=int(arr_add_days)) for arr_add_days in arr_add_days_list]
    arr_dates_list = [arr_date.strftime("%m-%d-%Y") for arr_date in arr_dates_list]
    
    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
    airlines_list = airlines_list[1:]
    
    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text.split("'") for value in prices]
    price_list = list(itertools.chain(*price_list))
    
    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
    
    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
    
    #layovers
    everything = browser.find_elements_by_xpath("//div[@class='secondary-content no-wrap']")
    to_parse = [value.text for value in everything]
    for value in to_parse:
        parsed = value.split('\n')
        departure_airport_code.append(parsed[1][0:3])
        arrival_airport_code.append(parsed[-1][-3:])
        if len(parsed) > 4:
            layovers_list.append(parsed[2])
        else:
            layovers_list.append("")
    
    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_date'] = arr_dates_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'flight_duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'num_stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'ticket_price'] = price_list[i]
            df.loc[i, 'ticket_price_time'] = str(current_date + '---' + current_time)
        except Exception as e:
            pass
        try:
            df.loc[i, 'departure_airport_code'] = departure_airport_code[i]
            df.loc[i, 'arrival_airport_code'] = arrival_airport_code[i]
        except Exception as e:
            pass

In [ ]:
def get_proxy(link):
    url = "https://www.sslproxies.org/"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    proxies_list = list(map(lambda x: x[0]+':'+x[1], list(zip(map(lambda x: x.text, soup.findAll('td')[::8]), map(lambda x: x.text, soup.findAll('td')[1::8])))))
    while 1:
        try:
            selected_ip = choice(proxies_list)
            proxy = {'https': selected_ip, 'http': selected_ip}
            headers = {'User-Agent': ua.random}
            print('Using proxy:{}'.format(proxy))
            r = requests.request('get', link, proxies=proxy, headers=headers, timeout=5)
            break
        except:
            pass
        
    return proxy
df = pd.DataFrame()
def compile_data(dep_date):
    global df
    global dep_times_list
    global arr_add_days_list
    global arr_dates_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list
    global departure_airport_code
    global arrival_airport_code
    
    dep_times_list = []
    arr_times_list = []
    arr_add_days_list = []
    arr_dates_list = []
    airlines_list = []
    price_list = []
    durations_list = []
    stops_list = []
    layovers_list = []
    departure_airport_code = []
    arrival_airport_code = []
    
    #departure times, arrival dates, arrival times
    everything = browser.find_elements_by_xpath("//div[@class='primary-content no-wrap custom-primary-padding']")
    to_parse = [value.text for value in everything]
for value in to_parse:
        parsed = value.split(' ')
        parsed[0] = parsed[0][:-2]
        dep_times_list.append(parsed[0])
        arr_times_list.append(parsed[1])
        if len(parsed) > 2:
            parsed[2] = parsed[2][:2]
            arr_add_days_list.append(int(parsed[2]))
        else:
            arr_add_days_list.append(0)
    
    arr_dates_list = [dep_date + timedelta(days=int(arr_add_days)) for arr_add_days in arr_add_days_list]
    arr_dates_list = [arr_date.strftime("%m-%d-%Y") for arr_date in arr_dates_list]
    
    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
    airlines_list = airlines_list[1:]
    
    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text.split("'") for value in prices]
    price_list = list(itertools.chain(*price_list))
    
    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
    
    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
    
    #layovers
    everything = browser.find_elements_by_xpath("//div[@class='secondary-content no-wrap']")
    to_parse = [value.text for value in everything]
    for value in to_parse:
        parsed = value.split('\n')
        departure_airport_code.append(parsed[1][0:3])
        arrival_airport_code.append(parsed[-1][-3:])
        if len(parsed) > 4:
            layovers_list.append(parsed[2])
        else:
            layovers_list.append("")
    
    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_date'] = arr_dates_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'flight_duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'num_stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'ticket_price'] = price_list[i]
            df.loc[i, 'ticket_price_time'] = str(current_date + '---' + current_time)
        except Exception as e:
            pass
        try:
            df.loc[i, 'departure_airport_code'] = departure_airport_code[i]
            df.loc[i, 'arrival_airport_code'] = arrival_airport_code[i]
        except Exception as e:
            pass
#skip a file if it's being scraped                           
files_in_path = [f for f in listdir(current_path+'/scraped data') if isfile(join(current_path+'/scraped data', f))]
Airports = pd.read_excel('airports.csv')
dep_airports = Airports['departure_airport'] 
arrival_airports = Airports['arrival_airport']
dep_year = 2020
dep_mon = 5
dep_day = 1
Num_days_scraped = 31
dates = [datetime.datetime(dep_year, dep_mon, dep_day+i) for i in range(Num_days_scraped)]
DRIVER_BIN = os.path.join(current_path, "geckodriver")
link = "https://www.expedia.com"
proxy = get_proxy(link)
userAgent = ua.random
Firefox_options = webdriver.FirefoxOptions()
Firefox_options.add_argument('--proxy-server=%s' % proxy)
Firefox_options.add_argument(f'user-agent={userAgent}')
browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)
i = 0
for dep_airport in dep_airports:   
    for arrival_airport in arrival_airports:
        flag = 1
        if arrival_airport != dep_airport:
            for dep_date in dates:
                filename = dep_airport + '_' + arrival_airport + '_' + dep_date.strftime("%m-%d-%Y") +'.csv' 
                if flag == 1 and filename not in files_in_path:      
                    try:
                        if i%3==0: #change a proxy if used 3 times
                            browser.close()
                            proxy = get_proxy(link)
                            userAgent = ua.random
                            Firefox_options = webdriver.FirefoxOptions()
                            Firefox_options.add_argument('--proxy-server=%s' % proxy)
                            Firefox_options.add_argument(f'user-agent={userAgent}')
                            browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)
                            
                        url = "https://www.expedia.com/Flights-Search?trip=oneway&leg1=from:{0},to:{1},departure:{2}TANYT&passengers=adults:1,children:0,seniors:0,infantinlap:Y&options=cabinclass%3Aeconomy&mode=search&origref=www.expedia.com".format(dep_airport,arrival_airport,dep_date.strftime("%m/%d/%Y"))
                        browser.get(url)
                        time.sleep(random.uniform(5,8))
                        
                        compile_data(dep_date)
                        if len(df) != 0:
                            #save values to excel
                            df['id'] = [(dep_airport + arrival_airport + dep_date.strftime("%m-%d-%Y") + '_' + str(i)) for i in range(1,len(df)+1)] 
                            df['departure_date'] = dep_date.strftime("%m-%d-%Y")
                            df['departure_airport_code_original'] = dep_airport
                            df['arrival_airport_code_original'] = arrival_airport
                            df = df[['id',
                                     'departure_date',
                                     'departure_airport_code',
                                     'departure_time',
                                     'arrival_date',
                                     'arrival_time',
                                     'airline',
                                     'arrival_airport_code',
                                     'flight_duration',
                                     'num_stops',
                                     'layovers',
                                     'ticket_price',
                                     'ticket_price_time',
                                  'departure_airport_code_original',
                                   'arrival_airport_code_original']]
                            
                            # Create a Pandas Excel writer using XlsxWriter as the engine.
                            filename = df['departure_airport_code'][0] + '_' + df['arrival_airport_code'][0] + '_' + str(dep_date.strftime("%m-%d-%Y")) +'.csv'
                            df.to_csv('scraped data/' + filename)
                            i += 1
                            time.sleep(random.uniform(0,3))
                        else:
                            flag = 0
                    except:
                        if KeyboardInterrupt:
                            print('Interrupted')
                            try:
                                sys.exit(0)
                            except SystemExit:
                                os._exit(0)
                        else:
                            browser.close()
                            proxy = get_proxy(link)
                            userAgent = ua.random
                            Firefox_options = webdriver.FirefoxOptions()
                            Firefox_options.add_argument('--proxy-server=%s' % proxy)
                            Firefox_options.add_argument(f'user-agent={userAgent}')
                            browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)